# ZFEL SASE

Basic example of a SASE simulation.

In [ ]:
import sys
sys.path.append('/Users/nneveu/github/zfel')
# (optional) installation
# !pip install git+https://github.com/slaclab/zfel.git@devel

In [ ]:
from zfel import sase1d
import zfel.plot

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## Input 

In [ ]:
sase_input = dict(
    npart   = 512,                       # n-macro-particles per bucket 
    s_steps = 200,                      # n-sample points along bunch length
    z_steps = 200,                      # n-sample points along undulator
    energy  = 4313.34e6,                # electron energy [eV]
    eSpread = 0,                       # relative rms energy spread [1]
    emitN   = 1.2e-6,                    # normalized transverse emittance [m-rad]
    currentMax = 3400,                   # peak current [Ampere]
    beta = 26,                          # mean beta [meter]
    unduPeriod = 0.03,                 # undulator period [meter]
    unduK = 3.5 ,          # undulator parameter, K [1], array could taper. 
    unduL = 70,                         # length of undulator [meter]
    radWavelength=None,                 # Will calculate based on resonance condition for unduK[0]
    random_seed=31,                     # for reproducibility
    particle_position=None, #np.genfromtxt('./Inputs/particle_position.csv', delimiter=',') # or None,
    hist_rule='square-root',             # 'square-root' or 'sturges' or 'rice-rule' or 'self-design', number \
                                       #  of intervals to generate the histogram of eta value in a bucket
    iopt='sase',
    P0 = 0                            # small seed input power [W]
)

## Intermediate parameters 

These are used internally

In [ ]:
params = sase1d.params_calc(**sase_input)
params.keys()

## Run

In [ ]:
%%time
output = sase1d.sase(sase_input)

In [ ]:
output.keys()

## Plot output

In [ ]:
plt.ylabel('Power (GW)')
plt.xlabel('z (m)')
plt.plot(output['z'], output['power_z']/1e9)

In [ ]:
# Power evolution. z vs slice
plt.imshow( (output['Er']**2 + output['Ei']**2).T, origin='lower')

In [ ]:
zfel.plot.plot_log_power_z(output)

In [ ]:
zfel.plot.plot_power_s(output)

In [ ]:
# plot spectrum
freq = output['freq']
spectrum = output['spectrum']
plt.plot(freq,spectrum[1:,-1])
plt.xlabel('energy (eV)')
plt.ylabel('spectrum (W/?)')

# Particle history of final slice

In [ ]:
theta = output['theta_final_slice_history']
eta = output['eta_final_slice_history']
_, n_step = theta.shape

# particles, step along z
theta.shape

In [ ]:
# Function to plot a single step
def plot_step(i):
    
    x = (theta[:,i] % 2*np.pi)/(2*np.pi) * 360
    y = eta[:,i]*1e3
    
    fig, ax = plt.subplots()
    
    ax.set_title(f'Final slice, Step {i}')
    ax.set_ylim(-6,6)
    ax.set_xlabel(r'$\theta\, (deg)$')
    ax.set_ylabel(r'$ \eta\, (10^{-3})$')
    ax.scatter(x, y, marker='.', color='black')

plot_step(n_step-1)

In [ ]:
# interactive
#from ipywidgets import interact
#import ipywidgets as widgets

#interact(plot_step, i=widgets.IntSlider(min=0, max=n_step-1, step=1, value=n_step//2));